IMPORTS & PREPRODUCTION

In [1]:
import os
import tensorflow as tf

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [2]:
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
import wandb
##wandb.init(project="LeNet_5_test", entity="hyunseokerikjung")

Data Processing

In [4]:
data = pd.read_csv('emnist-byclass-mapping.txt', sep=" ", header=None)
data.columns = ['Target','ASCII']

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [5]:
test_df = pd.read_csv('emnist-byclass-test.csv/emnist-byclass-test.csv')
test_df.rename(columns={'18':'Target'}, inplace=True)

# test_df=pd.merge(test_df, data, how='left', on='Target')
# test_df=test_df.drop(['Target','ASCII'],axis=1)
# test_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

In [6]:
train_df = pd.read_csv('emnist-byclass-train.csv/emnist-byclass-train.csv')
train_df.rename(columns={'35':'Target'}, inplace=True)

# train_df=pd.merge(train_df, data, how='left', on='Target')
# train_df=train_df.drop(['Target','ASCII'],axis=1)
# train_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

In [9]:
train_y = np.array(train_df.pop('Target'))
train_x = train_df
test_y = np.array(test_df.pop('Target'))
test_x = test_df

train_y = keras.utils.to_categorical(train_y)
test_y = keras.utils.to_categorical(test_y)

train_x = train_x / 255.0
test_x = test_x / 255.0

train_x = train_x.values.reshape([-1, 28, 28, 1])
test_x = test_x.values.reshape([-1, 28, 28, 1])

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

MODELS

Original LeNet_5

In [13]:
lenet_5_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(62, activation='softmax')
])

In [16]:
lenet_5_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [17]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

Epoch 1/100
17449/17449 [==============================] - 56s 3ms/step - loss: 0.5870 - accuracy: 0.8065 - val_loss: 0.4783 - val_accuracy: 0.8319
Epoch 2/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4565 - accuracy: 0.8389 - val_loss: 0.4584 - val_accuracy: 0.8394
Epoch 3/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4365 - accuracy: 0.8444 - val_loss: 0.4548 - val_accuracy: 0.8381
Epoch 4/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.4256 - accuracy: 0.8468 - val_loss: 0.4392 - val_accuracy: 0.8435
Epoch 5/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4182 - accuracy: 0.8490 - val_loss: 0.4350 - val_accuracy: 0.8454
Epoch 6/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.4133 - accuracy: 0.8503 - val_loss: 0.4357 - val_accuracy: 0.8454
Epoch 7/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4089 - accuracy: 0.8513 - val_

In [18]:
lenet_5_model.evaluate(test_x, test_y)

3636/3636 [==============================] - 6s 2ms/step - loss: 0.4194 - accuracy: 0.8510


[0.41943299770355225, 0.8510342240333557]

LeNet_5_SeLU

SeLU performs better(speed & accuracy)

In [9]:
keras.backend.clear_session()

lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.AveragePooling2D(),
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Flatten(),   
    keras.layers.Dense(84, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_selu_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

lenet_5_selu_model.evaluate(test_x, test_y)

Epoch 1/100
17449/17449 [==============================] - 56s 3ms/step - loss: 0.5854 - accuracy: 0.8044 - val_loss: 0.4761 - val_accuracy: 0.8315
Epoch 2/100
17449/17449 [==============================] - 54s 3ms/step - loss: 0.4568 - accuracy: 0.8375 - val_loss: 0.4583 - val_accuracy: 0.8375
Epoch 3/100
17449/17449 [==============================] - 54s 3ms/step - loss: 0.4356 - accuracy: 0.8436 - val_loss: 0.4438 - val_accuracy: 0.8408
Epoch 4/100
17449/17449 [==============================] - 55s 3ms/step - loss: 0.4244 - accuracy: 0.8468 - val_loss: 0.4442 - val_accuracy: 0.8469
Epoch 5/100
17449/17449 [==============================] - 56s 3ms/step - loss: 0.4156 - accuracy: 0.8492 - val_loss: 0.4279 - val_accuracy: 0.8490
Epoch 6/100
17449/17449 [==============================] - 57s 3ms/step - loss: 0.4105 - accuracy: 0.8508 - val_loss: 0.4271 - val_accuracy: 0.8450
Epoch 7/100
17449/17449 [==============================] - 56s 3ms/step - loss: 0.4061 - accuracy: 0.8518 - val_

[0.40906521677970886, 0.8539571166038513]

He Yanmei SeLU Lenet_5 model

Lowers amount of parameters = lower inference time, less stable model

In [7]:
keras.backend.clear_session()

lenet_5_improved_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=2, activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.Conv2D(64, kernel_size=3, strides=2, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.Conv2D(32, kernel_size=1, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

lenet_5_improved_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

lenet_5_improved_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb])

lenet_5_improved_model.evaluate(test_x, test_y)

Epoch 1/100
17449/17449 [==============================] - 59s 3ms/step - loss: 1.0427 - accuracy: 0.6946 - val_loss: 0.6979 - val_accuracy: 0.7794
Epoch 2/100
17449/17449 [==============================] - 52s 3ms/step - loss: 0.6480 - accuracy: 0.7906 - val_loss: 0.6001 - val_accuracy: 0.8030
Epoch 3/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.5862 - accuracy: 0.8074 - val_loss: 0.5815 - val_accuracy: 0.8102
Epoch 4/100
17449/17449 [==============================] - 52s 3ms/step - loss: 0.5567 - accuracy: 0.8155 - val_loss: 0.5578 - val_accuracy: 0.8155
Epoch 5/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.5374 - accuracy: 0.8207 - val_loss: 0.5401 - val_accuracy: 0.8202
Epoch 6/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.5239 - accuracy: 0.8247 - val_loss: 0.5326 - val_accuracy: 0.8222
Epoch 7/100
17449/17449 [==============================] - 53s 3ms/step - loss: 0.5149 - accuracy: 0.8263 - val_

[0.46336913108825684, 0.8403225541114807]

Optimizing *without* touching the data

With dropout = 86% test result

In [11]:
keras.backend.clear_session()

my_lenet_5_selu_model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, strides=1,  activation="selu", kernel_initializer="lecun_normal", input_shape=train_x[0].shape, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, kernel_size=3, strides=1, activation="selu", kernel_initializer="lecun_normal", padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(62, activation='softmax')
])

my_lenet_5_selu_model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_lenet5_emnist_model.h5", save_best_only=True)

my_lenet_5_selu_model.fit(train_x, train_y, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

my_lenet_5_selu_model.evaluate(test_x, test_y)

Epoch 1/100
 1466/17449 [=>............................] - ETA: 58s - loss: 2.4865 - accuracy: 0.3861

KeyboardInterrupt: 